In [1]:
from pymatgen import MPRester
import pprint

api_key='QBUZAI2ExwdstKO7'

mpr = MPRester(api_key)
all_mpids = mpr.query({}, ["material_id", "elements", "e_above_hull"])

In [2]:
print 'Number of materials in database:', len(all_mpids) # initially 67561 in db
print all_mpids[8464]

Number of materials in database: 67973
{u'elements': [u'I', u'N', u'Zr'], u'material_id': u'mp-23052', u'e_above_hull': 0}


In [3]:
# delete all with e_above_hull > 0.2 and if contains any Noble gases
noble_gases = ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn']

for mat in all_mpids:
    if bool(set(mat['elements']) & set(noble_gases)):
        all_mpids.remove(mat)
    elif mat['e_above_hull'] > 0.2:
        all_mpids.remove(mat)
        
print 'Number of materials in database < 0.2 eV and does not contain noble gases:', len(all_mpids)

Number of materials in database < 0.2 eV and does not contain noble gases: 62601


## clean out gases/liquids at standard

In [13]:
filter(lambda m: m['elements'] == ['Hg'], all_mpids)

[{u'e_above_hull': 0.0021126499999999937,
  u'elements': [u'Hg'],
  u'material_id': u'mp-569360'},
 {u'e_above_hull': 0.01239585999999998,
  u'elements': [u'Hg'],
  u'material_id': u'mp-753304'},
 {u'e_above_hull': 0.0030844266666666398,
  u'elements': [u'Hg'],
  u'material_id': u'mp-10861'},
 {u'e_above_hull': 0.002392559999999988,
  u'elements': [u'Hg'],
  u'material_id': u'mp-1017981'},
 {u'e_above_hull': 0.020461740000000006,
  u'elements': [u'Hg'],
  u'material_id': u'mp-982872'},
 {u'e_above_hull': 0.02432036999999998,
  u'elements': [u'Hg'],
  u'material_id': u'mp-121'},
 {u'e_above_hull': 0.08616251,
  u'elements': [u'Hg'],
  u'material_id': u'mp-611219'},
 {u'e_above_hull': 0.007525730000000008,
  u'elements': [u'Hg'],
  u'material_id': u'mp-569289'},
 {u'e_above_hull': 0.0008712927586206898,
  u'elements': [u'Hg'],
  u'material_id': u'mp-864900'},
 {u'e_above_hull': 0, u'elements': [u'Hg'], u'material_id': u'mp-975272'}]

In [11]:
mats_to_delete = ['mp-1010048','mp-998864']

for mat in all_mpids:
    if mat['material_id'] in mats_to_delete:
        all_mpids.remove(mat)
        print mat['material_id'], 'deleted'

mp-998864 deleted
mp-1010048 deleted


In [14]:
print 'final count:', len(all_mpids)

final count: 62594


## Put into MongoDB

In [15]:
from pymongo import MongoClient
client = MongoClient()

In [23]:
db = client.structure_sets
print db.collection_names()

[u'mp_final', u'materials_project']


In [ ]:
result = db.mp_bv_final.delete_many({})
result.deleted_count
db.mp_bv_final.drop()

In [19]:
cursor = db.mp_final.find(limit=1)
print cursor.count()

67466


In [21]:
# mpr.get_structure_by_material_id('mp-656887')
# all_mpids[0]['material_id']
# s = mpr.get_structure_by_material_id(all_mpids[1]['material_id'])
# s_dict = s.as_dict()

count = 0

for mp_id in all_mpids:
    s = mpr.get_structure_by_material_id(mp_id['material_id']).as_dict()
    result = db.materials_project.insert_one(s)
    
    count +=1
    if count%100 == 0:
        print count, 'structures imported.'
    

100 structures imported.
200 structures imported.
300 structures imported.
400 structures imported.
500 structures imported.
600 structures imported.
700 structures imported.
800 structures imported.
900 structures imported.
1000 structures imported.
1100 structures imported.
1200 structures imported.
1300 structures imported.
1400 structures imported.
1500 structures imported.
1600 structures imported.
1700 structures imported.
1800 structures imported.
1900 structures imported.
2000 structures imported.
2100 structures imported.
2200 structures imported.
2300 structures imported.
2400 structures imported.
2500 structures imported.
2600 structures imported.
2700 structures imported.
2800 structures imported.
2900 structures imported.
3000 structures imported.
3100 structures imported.
3200 structures imported.
3300 structures imported.
3400 structures imported.
3500 structures imported.
3600 structures imported.
3700 structures imported.
3800 structures imported.
3900 structures impor

In [22]:
cursor = db.materials_project.find()

cursor.count()
# for document in cursor:
#     print(document)

62595

In [ ]:
from pymatgen.analysis.bond_valence import BV_PARAMS, ELECTRONEG
from pymatgen import Element, Structure
import pprint

In [ ]:
# mpr.get_structure_by_material_id(all_mpids[4298]['material_id'])
cursor = db.mp_bv_hull_final.find()
cursor.count()

In [ ]:
# Insert into mp_bv_final

count = 0
for doc in cursor:
    # make sure at least two different elements in the structure are found in our database
    matching_ELECTRONEG_els = []
    matching_BV_els = []
    for site in doc['sites']:
        el = Element(site['label'])
        if el in ELECTRONEG:
            if el not in matching_ELECTRONEG_els:
                matching_ELECTRONEG_els.append(el)
        elif el in list(BV_PARAMS.keys()):
            if el not in matching_BV_els:
                matching_BV_els.append(el)
            
    if len(matching_ELECTRONEG_els) > 0 and len(matching_BV_els) > 0:
        result = db.mp_bv_final.insert_one(doc)
        
    count +=1
    if count%100 == 0:
        print count, 'iterated through structures.'

In [ ]:
for doc in cursor:
    for specie in doc['sites']:
        print specie['label']
        el = Element(specie['label'])
        if el in ELECTRONEG or el in list(BV_PARAMS.keys()):
            print el.symbol
#     pprint.pprint(doc['sites'])

In [ ]:
cursor = db.mp_bv_final.find(limit=5)
cursor.count()

In [ ]:
# Insert from mp_final into mp_bv_hull_final
# get all structures that are <50meV/atom (stability)

# count = 0
# IT DOESN'T DO THE LAST n IN THE [0:n]
for mpid in all_mpids[count:]:
    e_above_hull = mpr.get_entry_by_material_id(mpid['material_id'],property_data=['e_above_hull']).as_dict()['data']['e_above_hull']
    s = mpr.get_structure_by_material_id(mpid['material_id'])
    num_atoms = s.composition.num_atoms
    if e_above_hull/num_atoms <= 0.5:
        result = db.mp_bv_hull_final.insert_one(s.as_dict())
        
    count +=1
    if count%100 == 0:
        print count, 'structures iterated through.'

In [ ]:
po = [0, 1, 2, 3, 4, 5]
for a in po[0:5]:
    print a
    
count
    

In [ ]:
all_mpids[0]['material_id']

In [ ]:
cursor = db.mp_bv_final.find() 
cursor.count()

In [ ]:
cursor = db.mp_bv_final.find(limit=1)
for doc in cursor:
    print doc['_id']

In [ ]:
cursor = db.mp_bv_hull_final.find() 
cursor.count()